# Importing Libraries

In [2]:
!pip install pymatgen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.0/809.0 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 43.8 MB/s eta 0:00:00
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=43549 sha256=a810340615a0ca2bedacd2171b8b388be50f2bfbf2ae7cea7d40c6ef6977f39e
  Stored in directory: /root/.cache/pip/wheels/16/fb/76/306387739cf9d53b1c39b0c8aadbbb17dc05f256756d8fd915
Successfully built bibtexparser


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
import pymatgen.core as mg
import pickle

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score

# At the first step, a dataset of HEAs and elemental properties are used to create a dataset

In [9]:
#df = pd.read_csv('dataset10387_70.csv', encoding="iso-8859-1")

url = "https://raw.githubusercontent.com/SergeiVKalinin/ML_HEAs_Phase_Dataset/refs/heads/main/Dataset/dataset10387_70.csv"
df = pd.read_csv(url, encoding="iso-8859-1")
NAME_HEAs = list (df['Alloy'])

#%% creating an empty dataframe with columns of elements and row of alloys
element_symbols = ['Num_el','H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
                   'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca',
                   'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
                   'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr',
                   'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
                   'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
                   'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
                   'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
                   'Tl', 'Pb', 'Bi', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm',
                   'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg',
                   'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv',
                   'Ts', 'Og']

# Create an empty dataframe with columns named after element symbols
df2 = pd.DataFrame(columns=element_symbols)
print('Creating an Empty dataframe')
for s in tqdm(range (len(NAME_HEAs))):
    df2.loc[s] = [0]*len(element_symbols)


df2.index = NAME_HEAs

#%% to fill the above data frame
print('filling the dataframe regarding the composition of alloys')
i = 0
for name in tqdm(NAME_HEAs):
    # print(name)

    HEAs = mg.Composition(name)
    sym = []
    frac = []

    for i in range (len(HEAs)):
        el = HEAs.elements[i].symbol
        el_frac = HEAs.get_atomic_fraction(el)
        sym.append(el)
        frac.append(el_frac)

    for j in range (len(sym)):
        df2.loc[name, 'Num_el'] = len(sym)
        df2.loc[name, sym[j]] = frac[j]
#%%
cols_with_all_zeros = df2.columns[(df2 == 0).all()] #delete columns with zero values
df2 = df2.drop(cols_with_all_zeros, axis=1)
df2 = df2.iloc[:, :-5] #delete columns with none values
#%%
class Read_prepare_data:
    def __init__(self, name='elemental_propertieses.csv'):
        self.df = pd.read_csv(name)
        # self.df.drop('Unnamed: 0',inplace=True, axis=1)
        # self.df.sort_values(by=['atomic_number'], ascending=True, inplace=True, ignore_index=True)
        self.df.set_index('symbol', inplace=True)
    def list_att(self):
        return self.df.columns.tolist()

    def keep_att(self, list_keep): #When the number of attributes or discroptors is huge we just keep some of them
        self.df = self.df[list_keep]
        return  self.df.dropna()

    def drop_att(self, list_drop):

        for i in list_drop:
            self.df.drop(i, inplace=True, axis=1)

        return self.df

#%%
obj = Read_prepare_data()
att = obj.list_att()
my_choice2 = ['VEC', 'Pauling_EN', 'Melting_point_K','DFT_LDA_Etot',
'outer_shell_electrons', 'no_of_valence_electrons',
              'Atomic_radius_calculated', 'Atomic_weight']
df_element_attribute = obj.keep_att(my_choice2)
#%%
corr_matrix = df_element_attribute.corr()

#%%
class Mixture:
    def __init__(self, name, ATT, df):
        self.HEAs = mg.Composition(name)
        self.sym = []
        self.frac = []
        self.df = df
        self.ATT = ATT

        for i in range (len(self.HEAs)):
            self.el1 = self.HEAs.elements[i].symbol
            self.el1_frac = self.HEAs.get_atomic_fraction(self.el1)

            self.sym.append(self.el1)
            self.frac.append(self.el1_frac)

        self.b = []
        for m in self.sym:
            self.condition = self.df.index == m
            self.indices = self.df.index[self.condition]
            self.var = self.df.loc[self.indices, self.ATT]
            self.var2 = pd.Series.to_numpy(self.var)
            self.var2 = self.var2[0]

            self.b.append(self.var2)

    def mix(self):
        self.b = np.array(self.b)
        self.frac = np.array(self.frac)

        self.final = np.dot(self.b, self.frac)
        return self.final
#%%
att_name = list(df_element_attribute.columns)
#%%
print("Applying rule of mixture")
whole = []
i=0
for Name in tqdm(NAME_HEAs):
    #print(Name)
    #i=i+1
    #print(i)

    for at in att_name:
        ex = Mixture(Name, at, df_element_attribute)
        tmp = ex.mix()
        whole.append(tmp)
whole = np.reshape(whole, (len(NAME_HEAs), len(att_name)))
data = pd.DataFrame(whole, index = NAME_HEAs, columns=att_name)

#%%
#sqrt(sigma(i-ave)**2)
lst_difs = ['Pauling_EN']

for ii, lst_dif in enumerate(lst_difs):
    ind = list(data.index)
    dif_lst = []
    for iterator in ind:

        av = data.loc[iterator, lst_dif]
        if isinstance(av, pd.core.series.Series):
            av = av.iloc[0]
        sum_power_two_mul_frac = 0
        com = mg.Composition(iterator)
        for el in range (len(com)):
            symb = com.elements[el].symbol
            fra  = com.get_atomic_fraction(symb)
            i_th_att = df_element_attribute.loc[symb, lst_dif]
            power_two = (i_th_att-av)**2
            power_two_mul_frac = fra * power_two
            sum_power_two_mul_frac+= power_two_mul_frac

        final = np.sqrt(sum_power_two_mul_frac)
        dif_lst.append(final)


    data[lst_dif+'_div'] = dif_lst


#%% Entropy
ind = list(data.index)
entropy = 0
entropy_list = []
for iterator in ind:
    com = mg.Composition(iterator)
    entropy = 0
    for el in range (len(com)):
        symb = com.elements[el].symbol
        fra  = com.get_atomic_fraction(symb)
        entropy += -8.314*fra * ((np.log(fra)))
    entropy_list.append(entropy)


data['entropy'] = entropy_list

#%%
#sqrt(sigma(1-i/ave)**2)
lst_difs = ['Atomic_radius_calculated']

for ii, lst_dif in enumerate(lst_difs):
    ind = list(data.index)
    dif_lst = []
    for iterator in ind:

        av = data.loc[iterator, lst_dif]
        if isinstance(av, pd.core.series.Series):
            av = av.iloc[0]
        sum_power_two_mul_frac = 0
        com = mg.Composition(iterator)
        for el in range (len(com)):
            symb = com.elements[el].symbol
            fra  = com.get_atomic_fraction(symb)
            i_th_att = df_element_attribute.loc[symb, lst_dif]
            power_two = (1.0-i_th_att/av)**2
            power_two_mul_frac = fra * power_two
            sum_power_two_mul_frac+= power_two_mul_frac

        final = np.sqrt(sum_power_two_mul_frac)
        dif_lst.append(final)


    data[lst_dif+'_dif'] = dif_lst


#%% mixing enthalpy 4Hcicj
ind = list(data.index)

df_enthalpy = pd.read_csv('Enthalpy_mix.csv')
df_enthalpy.set_index('symbol', inplace=True)
df_enthalpy.drop('H',inplace=True, axis=1)

mix_enthalpy = []
for iterator in ind:
    #print(iterator)
    com = mg.Composition(iterator)

    symb = []
    frac = []
    for i in range (len(com)):
        sym = com.elements[i].symbol
        # el1_frac = com.get_atomic_fraction(sym)
        symb.append(sym)
        # frac.append(el1_frac)
    agg = 0

    for k in symb:
        for n in symb:
            if k==n:
                continue
            else:
                mix = df_enthalpy.loc[k,n]
                k_frac = com.get_atomic_fraction(k)
                n_frac = com.get_atomic_fraction(n)
                agg += mix * k_frac * n_frac

    mix_enthalpy.append(2*agg)



data['Enthalpy'] = mix_enthalpy
#%%

df_concatenated = pd.concat([df2, data], axis=1)
#%%

y = df["Phase"].copy()
y = y.to_frame()
data_rest = df_concatenated.reset_index()
data_rest['Geo'] = data_rest['entropy'] / (data_rest['Atomic_radius_calculated_dif'])**2
data_final = pd.concat([data_rest, y], axis=1)
data_final['Atomic_radius_calculated_dif'] = data_final['Atomic_radius_calculated_dif']*100
data_final['E_per_el'] = data_final['DFT_LDA_Etot']/data_final['Num_el']

KeyError: 'Alloy'

In [10]:
df

,Unnamed: 0,index,Num_el,Li,Be,B,C,N,Na,Mg,...,no_of_valence_electrons,Atomic_radius_calculated,Atomic_weight,Pauling_EN_div,entropy,Atomic_radius_calculated_dif,Enthalpy,Geo,Phase,E_per_el
0,0,AlAuCoCrCuNi,6,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0000,...,10.666667,1.506667,76.186167,0.302802,14.896688,11.759011,-10.000000,1077.328493,FCC+IM,-657.433169
1,1,AlCoCrCu0.5Fe,5,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0000,...,7.000000,1.476667,50.118000,0.116978,13.145213,11.545564,-8.197531,986.137229,FCC+IM,-210.818803
2,2,AlCoCrCuFe,5,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0000,...,7.400000,1.474000,51.460800,0.118423,13.380867,10.986313,-5.920000,1108.613795,FCC+IM,-222.492640
3,3,AlCoCrCuFeMnNi,7,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0000,...,7.714286,1.495714,52.990714,0.140378,16.178297,9.674466,-6.367347,1728.537500,FCC+IM,-167.680522
4,4,Al0.125CoCrCuFeMnNiTiV,9,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0000,...,7.430769,1.588615,54.908892,0.148324,17.683355,7.155794,-9.520473,3453.416819,BCC+FCC,-131.434543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10382,10382,CrCuFeMn2Ni2,5,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0000,...,8.428571,1.552857,56.950143,0.154774,12.885254,4.652868,-0.489796,5951.843777,FCC,-264.256237
10383,10383,Cr2CuFe2MnNi,5,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0000,...,8.000000,1.570000,56.123286,0.128698,12.885254,4.742062,2.612245,5730.052415,BCC+FCC,-254.230343
10384,10384,CrFe1.5MnNi0.5,4,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0000,...,7.500000,1.588750,55.012750,0.130839,10.981866,3.395947,-2.125000,9522.574388,BCC+IM,-302.181831
10385,10385,Al0.5BCoCrCuFeNi,7,0.00000,0.0,0.153846,0.0,0.0,0.0,0.0000,...,7.461538,1.406154,48.202615,0.129139,16.005458,18.142967,-11.881657,486.240895,IM,-153.221596


# Droping Duplicate

In [8]:
data = data_final
df = data.iloc[:, :-3].join(data.iloc[:, -1:].join(data.iloc[:, -2:-1]))
df.drop_duplicates(keep='first', inplace=True) # droping duplicates

NameError: name 'data_final' is not defined

In [ ]:
df.shape

(7590, 69)

# Droping data with inconsistant reports of phase

In [ ]:
duplicates = df[df.iloc[:, :-1].duplicated(keep=False)]
grouped = duplicates.groupby(list(duplicates.iloc[:, :-1].columns))
mask = grouped['Phase'].nunique() > 1
indices_to_drop = mask[mask].index
df.drop(df.index[df.iloc[:, :-1].apply(tuple, axis=1).isin(indices_to_drop)], inplace=True)

output = df['Phase'].to_frame()
df.drop(df.columns[-1], axis=1, inplace=True) # drop phase
df['Omega'] = df['Melting_point_K'] * df['entropy'] / abs(df['Enthalpy'])
df = pd.concat([df, output], axis=1)

# Anomoly detection

In [ ]:
df = df.iloc[:,-15:]
df = df.drop('Omega', axis=1)

In [ ]:
df.columns

Index(['VEC', 'Pauling_EN', 'Melting_point_K', 'DFT_LDA_Etot',
       'outer_shell_electrons', 'no_of_valence_electrons',
       'Atomic_radius_calculated', 'Atomic_weight', 'Pauling_EN_div',
       'entropy', 'Atomic_radius_calculated_dif', 'Enthalpy', 'E_per_el',
       'Phase'],
      dtype='object')

In [ ]:
output = df['Phase'].to_frame()
cat_list = output.index.tolist()
cat_num =  df['Phase'].value_counts()
#%%
def remove_outliers(data):
    data_encoded = pd.get_dummies(data, columns=['Phase'])

    model = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
    # Fit the model to the data
    model.fit(data_encoded)

    # Predict the outliers
    outliers = model.predict(data_encoded)

    # Filter the outliers from the original dataset
    data_filtered = data[~(outliers == -1)]
    cat_vars = ['Phase']
    filtered_cat_vars = data_filtered[cat_vars]
    filtered_data = pd.concat([data_filtered.drop(cat_vars, axis=1), filtered_cat_vars], axis=1)
    return filtered_data

BCC = df[df['Phase'] == 'BCC']
FCC = df[df['Phase'] == 'FCC']
FCC_BCC =  df[df['Phase'] == 'BCC+FCC']

BCC_IM = df[df['Phase'] == 'BCC+IM']
FCC_IM = df[df['Phase'] == 'FCC+IM']
FCC_BCC_IM =  df[df['Phase'] == 'BCC+FCC+IM']
IM =  df[df['Phase'] == 'IM']

AM =  df[df['Phase'] == 'AM']
BCC_AM =  df[df['Phase'] == 'BCC+AM']
FCC_AM =  df[df['Phase'] == 'FCC+AM']
FCC_BCC_AM =  df[df['Phase'] == 'BCC+FCC+AM']

dfs = [BCC, FCC, FCC_BCC, BCC_IM, FCC_IM, FCC_BCC_IM,IM, AM, BCC_AM,FCC_AM , FCC_BCC_AM ]

new_df = []
for dd in tqdm(dfs):
    dd = remove_outliers(dd)
    new_df.append(dd)
df = pd.concat(new_df, axis=0, ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:02<00:00,  4.08it/s]


In [ ]:
df.shape

(5677, 14)